In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("surajghuwalewala/ham1000-segmentation-and-classification")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2


In [2]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split


dataset_path = "/root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2"

# List files in the dataset directory
print("Dataset contents:", os.listdir(dataset_path))


Dataset contents: ['masks', 'GroundTruth.csv', 'images']


In [3]:
# Define dataset path (after downloading)
dataset_path = "/root/.cache/kagglehub/datasets/surajghuwalewala/ham1000-segmentation-and-classification/versions/2"

# Load the metadata file (assumed to be in CSV format)
metadata_path = os.path.join(dataset_path, "GroundTruth.csv")  # Adjust filename if needed
df = pd.read_csv(metadata_path)

# Ensure you have the required columns
print(df.head())

# Convert one-hot encoding to single class column
df['dx'] = df[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].idxmax(axis=1)

# Now apply label mapping
label_mapping = {"MEL": 0, "NV": 1, "BCC": 2, "AKIEC": 3, "BKL": 4, "DF": 5, "VASC": 6}
df['label'] = df['dx'].map(label_mapping)
df = df.dropna(subset=['label'])  # Remove rows with missing labels

# Define the image directory
image_dir = os.path.join(dataset_path, "images")  # Adjust if needed

# Splitting Data Stratified
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['dx'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['dx'], random_state=42)

# Function to move images to appropriate folders
def move_images(df, source_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for img_name in df['image']:
        src = os.path.join(source_dir, img_name + ".jpg")  # Adjust extension if needed
        dest = os.path.join(dest_dir, img_name + ".jpg")
        if os.path.exists(src):
            shutil.copy(src, dest)

# Create directories
move_images(train_df, image_dir, "dataset/train")
move_images(val_df, image_dir, "dataset/val")
move_images(test_df, image_dir, "dataset/test")

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")

          image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0024306  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0024307  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0024308  0.0  1.0  0.0    0.0  0.0  0.0   0.0
3  ISIC_0024309  0.0  1.0  0.0    0.0  0.0  0.0   0.0
4  ISIC_0024310  1.0  0.0  0.0    0.0  0.0  0.0   0.0
Train size: 7210, Val size: 802, Test size: 2003


In [4]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
import torch.nn as nn

classes = np.unique(train_df['label'])
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_df['label'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)


In [5]:
# Remove leakage by image ID
def remove_duplicates(df1, df2, key='image'):
    common = set(df1[key]).intersection(set(df2[key]))
    return df2[~df2[key].isin(common)]

test_df = remove_duplicates(train_df, test_df)
val_df = remove_duplicates(train_df, val_df)
test_df = remove_duplicates(val_df, test_df)

# Patient-level leakage removal
if 'patient_id' in df.columns:
    train_patients = set(train_df['patient_id'])
    val_df = val_df[~val_df['patient_id'].isin(train_patients)]
    test_df = test_df[~test_df['patient_id'].isin(train_patients)]
    test_df = test_df[~test_df['patient_id'].isin(set(val_df['patient_id']))]

In [6]:
# Create directories
def create_dirs(base='dataset'):
    for split in ['train', 'val', 'test']:
        for cls in label_mapping:
            path = os.path.join(base, split, cls)
            os.makedirs(path, exist_ok=True)

create_dirs()

In [7]:
# Move and check images
corrupt_images = []

def move_and_check(df, split, source_dir, dest_base):
    for _, row in df.iterrows():
        img_name = row['image'] + ".jpg"
        label = row['dx']
        src = os.path.join(source_dir, img_name)
        dest = os.path.join(dest_base, split, label, img_name)
        try:
            # Validate image
            with Image.open(src) as img:
                img.verify()  # Will raise error if corrupt
            shutil.copy(src, dest)
        except Exception as e:
            corrupt_images.append(img_name)

move_and_check(train_df, 'train', image_dir, 'dataset')
move_and_check(val_df, 'val', image_dir, 'dataset')
move_and_check(test_df, 'test', image_dir, 'dataset')

print(f"Total corrupt images found and skipped: {len(corrupt_images)}")


Total corrupt images found and skipped: 10015


In [8]:
import random
from PIL import Image
import os

# Image augmentation functions
def random_rotation(image):
    return image.rotate(random.uniform(-30, 30))

def random_flip(image):
    if random.random() > 0.5:
        return image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

def random_crop(image, output_size=(224, 224)):
    width, height = image.size
    left = random.randint(0, width // 4)
    top = random.randint(0, height // 4)
    right = width - random.randint(0, width // 4)
    bottom = height - random.randint(0, height // 4)
    return image.crop((left, top, right, bottom)).resize(output_size)


def preprocess_and_save(df, source_dir, dest_dir, image_column, label_column):
    processed_count = 0
    missing_count = 0
    error_count = 0

    for _, row in df.iterrows():
        img_name = row[image_column]
        label = str(row[label_column])  # e.g. '0', '1', ..., '6'
        class_dir = os.path.join(dest_dir, label)
        os.makedirs(class_dir, exist_ok=True)

        src = os.path.join(source_dir, img_name)
        dest = os.path.join(class_dir, img_name)

        # Check if file exists with extensions
        if not os.path.exists(src):
            if os.path.exists(src + ".jpg"):
                src += ".jpg"
                dest += ".jpg"
            elif os.path.exists(src + ".png"):
                src += ".png"
                dest += ".png"
            else:
                print(f"❌ Missing during preprocessing: {src}")
                missing_count += 1
                continue

        try:
            with Image.open(src) as img:
                img = img.convert("RGB")
                img.save(dest)
                processed_count += 1
        except Exception as e:
            print(f"❌ Error processing {src}: {e}")
            error_count += 1

    print(f"✅ Processed images: {processed_count}")
    print(f"❌ Missing images: {missing_count}")
    print(f"❌ Errors during processing: {error_count}")

# Re-run for the test set
preprocess_and_save(test_df, "dataset/test", "dataset/preprocessed_test", "image", "label")


# Apply preprocessing
preprocess_and_save(train_df, "dataset/train", "dataset/preprocessed_train", "image","label" )
preprocess_and_save(test_df, "dataset/test", "dataset/preprocessed_test", "image","label")
preprocess_and_save(val_df, "dataset/val", "dataset/preprocessed_val", "image","label")

print("Preprocessing complete.")


✅ Processed images: 2003
❌ Missing images: 0
❌ Errors during processing: 0
✅ Processed images: 7210
❌ Missing images: 0
❌ Errors during processing: 0
✅ Processed images: 2003
❌ Missing images: 0
❌ Errors during processing: 0
✅ Processed images: 802
❌ Missing images: 0
❌ Errors during processing: 0
Preprocessing complete.


In [9]:
import os
from collections import Counter

def count_images_per_class(base_dir):
    counts = {}
    for class_name in os.listdir(base_dir):
        class_dir = os.path.join(base_dir, class_name)
        if os.path.isdir(class_dir):
            counts[class_name] = len(os.listdir(class_dir))
    return counts

print(count_images_per_class("dataset/preprocessed_train"))


{'6': 102, '4': 791, '2': 370, '3': 236, '0': 801, '1': 4827, '5': 83}


In [10]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load pre-trained ResNet18 (only ~11.7 million parameters)
def build_model(num_classes):
    model = models.resnet18(pretrained=True)

    # Replace the final classification layer to match HAM10000 classes
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    return model

# Count total trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Build model
num_classes = 7  # MEL, NV, BCC, AKIEC, BKL, DF, VASC
model = build_model(num_classes).to(device)

# Print model summary
total_params = count_parameters(model)
print(f"✅ Total trainable parameters: {total_params:,}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Total trainable parameters: 11,180,103


In [11]:
import torch
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Evaluation function
def evaluate_model(model, dataloader, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Overall Accuracy
    acc = accuracy_score(all_labels, all_preds)
    print(f"\n✅ Test Accuracy: {acc:.4f}\n")

    # Classification Report (Precision, Recall, F1 per class)
    print("🔍 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("📊 Confusion Matrix")
    plt.tight_layout()
    plt.show()


In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet input size
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])


# Datasets
train_dataset = datasets.ImageFolder("dataset/preprocessed_train", transform=transform)
val_dataset = datasets.ImageFolder("dataset/preprocessed_val", transform=transform)
test_dataset = datasets.ImageFolder("dataset/preprocessed_test", transform=transform)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=5,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=5,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=5,pin_memory=True)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [15]:
# import torch.optim as optim

# def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=3):
#     best_val_loss = float('inf')
#     early_stop_counter = 0

#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0
#         correct = 0
#         total = 0

#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item() * images.size(0)
#             _, preds = torch.max(outputs, 1)
#             correct += (preds == labels).sum().item()
#             total += labels.size(0)

#         train_loss = running_loss / total
#         train_acc = correct / total

#         # Validation
#         model.eval()
#         val_loss = 0.0
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for images, labels in val_loader:
#                 images, labels = images.to(device), labels.to(device)
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item() * images.size(0)
#                 _, preds = torch.max(outputs, 1)
#                 correct += (preds == labels).sum().item()
#                 total += labels.size(0)

#         val_loss /= total
#         val_acc = correct / total

#         print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

#         # Early stopping
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             early_stop_counter = 0
#             torch.save(model.state_dict(), 'best_model.pth')
#             print("✅ Model saved!")
#         else:
#             early_stop_counter += 1
#             if early_stop_counter >= patience:
#                 print("⏹️ Early stopping triggered.")
#                 break

#     print("🎉 Training complete!")

# # Define optimizer and train the model
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, patience=5)


In [16]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from tqdm import tqdm

# # Training function optimized for speed with AMP and fewer syncs
# def train_model(model, train_loader, val_loader, num_epochs=10, accumulation_steps=4):
#     model.train()
#     optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
#     scaler = torch.cuda.amp.GradScaler()  # Mixed precision training
#     criterion = nn.CrossEntropyLoss()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0
#         optimizer.zero_grad(set_to_none=True)  # Reduce memory usage

#         for i, (inputs, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
#             inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

#             with torch.cuda.amp.autocast():  # Enable mixed precision
#                 outputs = model(inputs)
#                 loss = criterion(outputs, labels) / accumulation_steps  # Normalize loss

#             scaler.scale(loss).backward()

#             if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
#                 scaler.step(optimizer)
#                 scaler.update()
#                 optimizer.zero_grad(set_to_none=True)

#             running_loss += loss.item() * accumulation_steps

#         print(f"Epoch {epoch+1} Loss: {running_loss / len(train_loader):.4f}")

#         # Validation loop with minimal computation overhead
#         model.eval()
#         correct, total = 0, 0
#         with torch.no_grad():
#             for inputs, labels in val_loader:
#                 inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
#                 outputs = model(inputs)
#                 _, preds = torch.max(outputs, 1)
#                 correct += (preds == labels).sum().item()
#                 total += labels.size(0)

#         print(f"Validation Accuracy: {100 * correct / total:.2f}%")

#     print("Training complete.")

# # Call the function to start training
# train_model(model, train_loader, val_loader, num_epochs=10, accumulation_steps=4)


<ipython-input-16-d746877d9318>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Mixed precision training
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1/10:   0%|          | 0/451 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-16-d746877d9318>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` i

KeyboardInterrupt: 

In [ ]:
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# train_transform = A.Compose([
#     A.Resize(224, 224),
#     A.HorizontalFlip(),
#     A.RandomBrightnessContrast(),
#     A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15),
#     A.Normalize(),
#     ToTensorV2(),
# ])


In [ ]:
# from torchvision.datasets import ImageFolder
# import cv2

# class AlbumentationsImageFolder(ImageFolder):
#     def __init__(self, root, transform=None):
#         super().__init__(root)
#         self.albumentations_transform = transform

#     def __getitem__(self, index):
#         path, label = self.samples[index]
#         image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         if self.albumentations_transform:
#             image = self.albumentations_transform(image=image)['image']

#         return image, label


In [ ]:
# import time
# from torch.utils.data import DataLoader
# from tqdm import tqdm
# import cv2
# import os

# # Use your custom dataset
# # Ensure this dataset uses cv2 and albumentations as discussed
# train_dataset = AlbumentationsImageFolder(root='dataset/preprocessed_train', transform=train_transform)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)  # Set workers to 0 for clear profiling

# # Test just 1 batch to analyze bottlenecks
# print("🔍 Running bottleneck test on a single batch...\n")

# start_total = time.time()

# for i, (images, labels) in enumerate(train_loader):
#     if i >= 1:  # Only run one batch
#         break

#     print(f"\n✅ Loaded batch {i + 1}")
#     print(f"🧪 Images shape: {images.shape}")
#     print(f"🧪 Labels: {labels}")

# end_total = time.time()

# print(f"\n⏱️ Total time to load 1 batch: {end_total - start_total:.2f} seconds")


In [ ]:
# print("\n🔍 Profiling per-sample in dataset...\n")
# sample_times = []

# for idx in tqdm(range(10)):  # Test on just 10 samples
#     start = time.time()
#     image, label = train_dataset[idx]
#     end = time.time()
#     sample_times.append(end - start)

# avg_time = sum(sample_times) / len(sample_times)
# print(f"\n⏱️ Avg time per sample: {avg_time:.4f} seconds (~{1/avg_time:.2f} samples/sec)")


In [ ]:
# from torch.optim import Adam
# from torch.utils.data import DataLoader
# from tqdm import tqdm

# # Define train and val datasets
# # train_dataset = ImageFolder('dataset/preprocessed_train', transform=train_transform)
# # val_dataset = ImageFolder('dataset/preprocessed_val', transform=test_transforms)

# # train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
# # val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
# train_dataset = AlbumentationsImageFolder(root='dataset/preprocessed_train', transform=train_transform)
# val_dataset = AlbumentationsImageFolder(root='dataset/preprocessed_val', transform=test_transforms)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4,pin_memory=True)


# optimizer = Adam(model.parameters(), lr=1e-4)
# num_epochs = 10  # increase for better performance

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0

#     for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()

#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f"🔥 Epoch {epoch+1}, Training Loss: {running_loss / len(train_loader):.4f}")


In [ ]:
# Your DataLoader for the preprocessed test set
# Example: test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
test_dataset = ImageFolder(root='dataset/preprocessed_test', transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Define class labels (match your label mapping)
class_names = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"]

# Evaluate the model
evaluate_model(model, test_loader, class_names)


In [ ]:
#THIS IS WITHOUT BONUS
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()

        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True),
            )

        self.enc1 = conv_block(in_channels, 64)
        self.enc2 = conv_block(64, 128)
        self.enc3 = conv_block(128, 256)
        self.enc4 = conv_block(256, 512)

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = conv_block(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = conv_block(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = conv_block(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = conv_block(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = conv_block(128, 64)

        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.dec4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.dec3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.dec1(d1)

        return torch.sigmoid(self.out_conv(d1))


In [ ]:
# outputs_class, outputs_mask = model(images)

# # Classification loss
# loss_cls = criterion_class(outputs_class, labels)

# # Segmentation loss
# loss_seg = criterion_seg(outputs_mask, masks)

# # Total loss (you can tune the weights)
# loss = loss_cls + loss_seg


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_names = os.listdir(image_dir)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        mask_path = os.path.join(self.mask_dir, self.image_names[idx].replace(".jpg", "_mask.png"))  # adjust as needed

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # binary mask

        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image = augmented['image']
            mask = augmented['mask']

        return image, mask


In [ ]:
import torch
import numpy as np

# Dice Coefficient: 2 * |A ∩ B| / (|A| + |B|)
def dice_score(pred, target, smooth=1e-6):
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

# IoU: |A ∩ B| / |A ∪ B|
def iou_score(pred, target, smooth=1e-6):
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + smooth) / (union + smooth)


In [ ]:
def evaluate_model(model, dataloader, device):
    model.eval()
    dice_scores = []
    iou_scores = []

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            preds = (outputs > 0.5).float()  # Threshold for binary masks
            #print(preds.unique(), true.unique())
            for pred, true in zip(preds, masks):
                pred = pred.view(-1)
                true = true.view(-1)
                # Normalize to float32 binary masks
                pred = pred.float()
                true = (true > 0.5).float()

                dice = dice_score(pred, true)
                iou = iou_score(pred, true)
                dice_scores.append(dice.item())
                iou_scores.append(iou.item())

    avg_dice = np.mean(dice_scores)
    avg_iou = np.mean(iou_scores)

    print(f"\n✅ Test Dice Coefficient: {avg_dice:.4f}")
    print(f"✅ Test IoU Score: {avg_iou:.4f}")
    return avg_dice, avg_iou
dice, iou = evaluate_model(model, test_loader, device)

#dice score is low, >0.7 is good, IoU score was too high,should be from 0-1, >0.5 is ok,, >0.8 is good
